# Wikidata items

In [1]:
!pip install sparqlwrapper

import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import re


## Query Wikidata function

The function takes a SPARQL query string as its argument. It will run the SPARQL query and return the result as a data frame

In [2]:
def query_wikidata(query):
    endpoint_url = "https://query.wikidata.org/sparql"

    def get_results(endpoint_url, query):
        user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
        # TODO adjust user agent; see https://w.wiki/CX6
        sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()


    results = get_results(endpoint_url, query)

    raw = pd.json_normalize(results["results"]["bindings"])

    df = raw.filter(regex="\.value$")
    df = df.rename(columns=lambda x: re.sub('\.value$','',x))
    
    if 'orcid' not in df.columns:
        df['orcid'] = None
    if 'au_dict_bio' not in df.columns:
        df['au_dict_bio'] = None
    
    # order columns so that they are always in the same order
    cols = ['item', 'itemLabel', 'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 'enc_au_sc_id', 'au_dict_bio',
            'yob', 'yod', 'wyb', 'wye']
    df = df[cols]
    
    return df

## SPARQL queries

Because wildcard search against item label in Wikidata are very slow and generally time out, we run a number of searches on the presence of relavant identifiers.

SPARQL queries courtesy of Mathias Dillen, Botanic Garden Meise: https://github.com/matdillen/STSM-wikidata-people/blob/master/collectormatching.Rmd. I have changed them slightly by removing the identifier the presence of which is queried on from the SELECT clause and adding the Harvard Index of Botanists ID (P6264), IPNI ID (P586) and IPNI Standard Form (P428). This way all queries return the same terms and no data will be lost when duplicates are removed. The added terms will be valuable for verifying matches later on.

Update 2020-04-14: Added Bloodhound Tracker ID (P6944) and Encyclopedia of Australian Science (P4228).

Update 2020-04-16: Added Australian Dictionary of Biography (P1907). Also tried Trove (P1315), but that query times out.

### Entomologists of the world (P5370)

In [3]:
queries = {}

queries['ento'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye 
  ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id ?au_dict_bio
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P5370 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid . }
  OPTIONAL { ?item wdt:P214 ?viaf . }
  OPTIONAL { ?item wdt:P213 ?isni . }
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P1907 ?au_dict_bio . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### BHL creators (P4081)

In [4]:
queries['bhl_creator'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye 
  ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id ?au_dict_bio
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P4081 ?id .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid . }
  OPTIONAL { ?item wdt:P214 ?viaf . }
  OPTIONAL { ?item wdt:P213 ?isni . }
  OPTIONAL { ?item wdt:P6264 ?harv }
  OPTIONAL { ?item wdt:P586 ?ipni }
  OPTIONAL { ?item wdt:P428 ?abbr }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P1907 ?au_dict_bio . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Harvard Index of Botanists ID (P6264)

In [5]:
queries['harvard_index'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye 
  ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id ?au_dict_bio
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P6264 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid . }
  OPTIONAL { ?item wdt:P214 ?viaf . }
  OPTIONAL { ?item wdt:P213 ?isni . }
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P1907 ?au_dict_bio . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Zoobank ID (P2006)

In [6]:
queries['zoobank_id'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye 
  ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id ?au_dict_bio
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P2006 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid .}
  OPTIONAL { ?item wdt:P214 ?viaf .}
  OPTIONAL { ?item wdt:P213 ?isni .}
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P1907 ?au_dict_bio . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### IPNI ID (P586)

In [7]:
queries['ipni_id'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye 
  ?harv ?ipni ?abbr  ?bloodhound_id ?enc_au_sc_id ?au_dict_bio
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P586 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid .}
  OPTIONAL { ?item wdt:P214 ?viaf .}
  OPTIONAL { ?item wdt:P213 ?isni .}
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P1907 ?au_dict_bio . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Wikispecies article

In [8]:
queries['wikispecies'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye 
  ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id ?au_dict_bio
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?id schema:about ?item ;
    schema:isPartOf <https://species.wikimedia.org/> .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
  OPTIONAL { ?item wdt:P496 ?orcid . }
  OPTIONAL { ?item wdt:P214 ?viaf . }
  OPTIONAL { ?item wdt:P213 ?isni . }
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P1907 ?au_dict_bio . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Bloodhound Tracker ID (P6944)

In [9]:
queries['bloodhound_id'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye 
  ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id ?au_dict_bio
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P6944 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid .}
  OPTIONAL { ?item wdt:P214 ?viaf .}
  OPTIONAL { ?item wdt:P213 ?isni .}
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P1907 ?au_dict_bio . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Encyclopedia of Australian Science ID (P4228)

In [10]:
queries['encyclopedia_australian_science_id'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye 
  ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id ?au_dict_bio
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P4228 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid .}
  OPTIONAL { ?item wdt:P214 ?viaf .}
  OPTIONAL { ?item wdt:P213 ?isni .}
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P1907 ?au_dict_bio . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

### Australian Dictionary of Biography (P1907)

In [11]:
queries['australian_dictionary_biography'] = """
SELECT DISTINCT ?item ?itemLabel ?orcid ?viaf ?isni ?yob ?yod ?fly ?wyb ?wye 
  ?harv ?ipni ?abbr ?bloodhound_id ?enc_au_sc_id ?au_dict_bio
WHERE {
  ?item wdt:P31 wd:Q5 .
  ?item wdt:P1907 ?id . 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" } 
  OPTIONAL { ?item wdt:P496 ?orcid . }
  OPTIONAL { ?item wdt:P214 ?viaf . }
  OPTIONAL { ?item wdt:P213 ?isni . }
  OPTIONAL { ?item wdt:P6264 ?harv . }
  OPTIONAL { ?item wdt:P586 ?ipni . }
  OPTIONAL { ?item wdt:P428 ?abbr . }
  OPTIONAL { ?item wdt:P6944 ?bloodhound_id . }
  OPTIONAL { ?item wdt:P4228 ?enc_au_sc_id . }
  OPTIONAL { ?item wdt:P1907 ?au_dict_bio . }
  OPTIONAL { ?item wdt:P569 ?dob . BIND(YEAR(?dob) as ?yob) }
  OPTIONAL { ?item wdt:P570 ?dod . BIND(YEAR(?dod) as ?yod) }
  OPTIONAL { ?item wdt:P1317 ?fl . BIND(YEAR(?fl) as ?fly) }
  OPTIONAL { ?item wdt:P2031 ?wpb . BIND(YEAR(?wpb) as ?wyb) }
  OPTIONAL { ?item wdt:P2032 ?wpe . BIND(YEAR(?wpe) as ?wye) }
}
"""

## Create the data frame

Run all the SPARQL requests, concatenate the results and drop duplicates

In [12]:
# Run the queries and create a list of data frames
frames = []
for key, query in queries.items():
    dfi = query_wikidata(query)
    print(key + ': ' + str(len(dfi.index)) + ' records')
    frames.append(dfi)

# Concatenate the dataframes from each SPARQL request
df = pd.concat(frames)

# Drop duplicates
df = df.drop_duplicates(subset=['item'])

df.head()

ento: 10664 records
bhl_creator: 24332 records
harvard_index: 23877 records
zoobank_id: 15226 records
ipni_id: 54701 records
wikispecies: 46509 records
bloodhound_id: 1639 records
encyclopedia_australian_science_id: 2842 records
australian_dictionary_biography: 13479 records


,item,itemLabel,orcid,viaf,isni,harv,ipni,abbr,bloodhound_id,enc_au_sc_id,au_dict_bio,yob,yod,wyb,wye
0,http://www.wikidata.org/entity/Q67329,Carl Gustav Jablonsky,NaN,24944037,0000 0000 5526 3001,NaN,NaN,NaN,NaN,NaN,NaN,1756,1787,NaN,NaN
1,http://www.wikidata.org/entity/Q68738,Hermann Müller,NaN,2532803,0000 0001 0837 1728,36129,6735-1,H.Müll.,NaN,NaN,NaN,1829,1883,NaN,NaN
2,http://www.wikidata.org/entity/Q66902,Hermann Lebert,NaN,27833384,0000 0001 2100 1924,67704,22162-1,Lebert,NaN,NaN,NaN,1813,1878,NaN,NaN
3,http://www.wikidata.org/entity/Q69552,Carl Julius Bernhard Börner,NaN,118457204,0000 0001 1085 9124,1763,12350-1,Börner,NaN,NaN,NaN,1880,1953,NaN,NaN
4,http://www.wikidata.org/entity/Q66379,Ludwig Bechstein,NaN,36914329,0000 0001 2278 3438,NaN,NaN,NaN,NaN,NaN,NaN,1801,1860,NaN,NaN


Add surname, initials and canonical string (`surname` + ', ' + `initials`) columns 

In [13]:
surname = []
initials = []
canonical = []

for i, item in df.iterrows():
    words = re.split('[ .]', item['itemLabel'])
    words = [string for string in words if string != ""]
    surname.append(words[-1])
    initials.append(".".join([x[0] for x in words[0:-1]]) + '.')
    canonical.append(words[-1] + ', ' + ".".join([x[0] for x in words[0:-1] if len(x) > 0]) + '.')
    
df['surname'] = surname
df['initials'] = initials
df['canonical_string'] = canonical
    
df.head()

,item,itemLabel,orcid,viaf,isni,harv,ipni,abbr,bloodhound_id,enc_au_sc_id,au_dict_bio,yob,yod,wyb,wye,surname,initials,canonical_string
0,http://www.wikidata.org/entity/Q67329,Carl Gustav Jablonsky,NaN,24944037,0000 0000 5526 3001,NaN,NaN,NaN,NaN,NaN,NaN,1756,1787,NaN,NaN,Jablonsky,C.G.,"Jablonsky, C.G."
1,http://www.wikidata.org/entity/Q68738,Hermann Müller,NaN,2532803,0000 0001 0837 1728,36129,6735-1,H.Müll.,NaN,NaN,NaN,1829,1883,NaN,NaN,Müller,H.,"Müller, H."
2,http://www.wikidata.org/entity/Q66902,Hermann Lebert,NaN,27833384,0000 0001 2100 1924,67704,22162-1,Lebert,NaN,NaN,NaN,1813,1878,NaN,NaN,Lebert,H.,"Lebert, H."
3,http://www.wikidata.org/entity/Q69552,Carl Julius Bernhard Börner,NaN,118457204,0000 0001 1085 9124,1763,12350-1,Börner,NaN,NaN,NaN,1880,1953,NaN,NaN,Börner,C.J.B.,"Börner, C.J.B."
4,http://www.wikidata.org/entity/Q66379,Ludwig Bechstein,NaN,36914329,0000 0001 2278 3438,NaN,NaN,NaN,NaN,NaN,NaN,1801,1860,NaN,NaN,Bechstein,L.,"Bechstein, L."


In [14]:
df = df[['item', 'itemLabel', 
        'surname', 'initials', 'canonical_string', 
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 'enc_au_sc_id', 'au_dict_bio',
        'yob', 'yod', 'wyb', 'wye']]
df.reset_index(drop=True, inplace=True)
df.head()

,item,itemLabel,surname,initials,canonical_string,orcid,viaf,isni,harv,ipni,abbr,bloodhound_id,enc_au_sc_id,au_dict_bio,yob,yod,wyb,wye
0,http://www.wikidata.org/entity/Q67329,Carl Gustav Jablonsky,Jablonsky,C.G.,"Jablonsky, C.G.",NaN,24944037,0000 0000 5526 3001,NaN,NaN,NaN,NaN,NaN,NaN,1756,1787,NaN,NaN
1,http://www.wikidata.org/entity/Q68738,Hermann Müller,Müller,H.,"Müller, H.",NaN,2532803,0000 0001 0837 1728,36129,6735-1,H.Müll.,NaN,NaN,NaN,1829,1883,NaN,NaN
2,http://www.wikidata.org/entity/Q66902,Hermann Lebert,Lebert,H.,"Lebert, H.",NaN,27833384,0000 0001 2100 1924,67704,22162-1,Lebert,NaN,NaN,NaN,1813,1878,NaN,NaN
3,http://www.wikidata.org/entity/Q69552,Carl Julius Bernhard Börner,Börner,C.J.B.,"Börner, C.J.B.",NaN,118457204,0000 0001 1085 9124,1763,12350-1,Börner,NaN,NaN,NaN,1880,1953,NaN,NaN
4,http://www.wikidata.org/entity/Q66379,Ludwig Bechstein,Bechstein,L.,"Bechstein, L.",NaN,36914329,0000 0001 2278 3438,NaN,NaN,NaN,NaN,NaN,NaN,1801,1860,NaN,NaN


In [28]:
wikidata_link = []
orcid_link = []
harv_link = []
ipni_link = []
bloodhound_link = []
enc_au_sc_link = []
au_dict_bio_link = []
for i, row in df.iterrows():
    wikidata_link.append(row['item'].replace('entity', 'wiki'))
    orcid = 'https://orcid.org/' + str(row['orcid']) if pd.notnull(row['orcid']) else None
    orcid_link.append(orcid)
    harv_link.append('https://kiki.huh.harvard.edu/databases/botanist_search.php?mode=details&id=' + str(row['harv']) if pd.notnull(row['harv']) else None)
    ipni_link.append('https://www.ipni.org/a/' + row['ipni'] if pd.notnull(row['ipni']) else None)
    bloodhound_link.append('https://bloodhound-tracker.net/' + row['bloodhound_id'] if pd.notnull(row['bloodhound_id']) else None)
    enc_au_sc_link.append('http://www.eoas.info/biogs/' + row['enc_au_sc_id'] if pd.notnull(row['enc_au_sc_id']) else None)
    au_dict_bio_link.append('http://adb.anu.edu.au/biography/' + row['au_dict_bio'] if pd.notnull(row['au_dict_bio']) else None)
    
df['wikidata_link'] = wikidata_link
df['orcid_link'] = orcid_link
df['harv_link'] = harv_link
df['ipni_link'] = ipni_link
df['bloodhound_link'] = bloodhound_link
df['enc_au_sc_link'] = enc_au_sc_link
df['au_dict_bio_link'] = au_dict_bio_link

df[df['au_dict_bio_link'].notnull()]

,item,itemLabel,surname,initials,canonical_string,orcid,viaf,isni,harv,ipni,...,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bloodhound_link,enc_au_sc_link,au_dict_bio_link
10,http://www.wikidata.org/entity/Q67471,Georg von Neumayer,Neumayer,G.v.,"Neumayer, G.v.",NaN,22442169,0000 0001 2330 9615,971,6934-1,...,1909,NaN,NaN,http://www.wikidata.org/wiki/Q67471,None,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6934-1,None,http://www.eoas.info/biogs/P000104b,http://adb.anu.edu.au/biography/neumayer-georg...
176,http://www.wikidata.org/entity/Q76851,Amalie Dietrich,Dietrich,A.,"Dietrich, A.",NaN,77113813,0000 0000 3219 7769,8587,NaN,...,1891,NaN,NaN,http://www.wikidata.org/wiki/Q76851,None,https://kiki.huh.harvard.edu/databases/botanis...,None,https://bloodhound-tracker.net/Q76851,http://www.eoas.info/biogs/P000366b,http://adb.anu.edu.au/biography/dietrich-amali...
312,http://www.wikidata.org/entity/Q89624,John Stanislaw Kubary,Kubary,J.S.,"Kubary, J.S.",NaN,5230890,0000 0000 7141 8050,15350,NaN,...,1896,NaN,NaN,http://www.wikidata.org/wiki/Q89624,None,https://kiki.huh.harvard.edu/databases/botanis...,None,None,http://www.eoas.info/biogs/P000125b,http://adb.anu.edu.au/biography/kubary-john-st...
473,http://www.wikidata.org/entity/Q106178,Richard Schomburgk,Schomburgk,R.,"Schomburgk, R.",NaN,30332468,0000 0000 8368 6685,NaN,9231-1,...,1891,NaN,NaN,http://www.wikidata.org/wiki/Q106178,None,None,https://www.ipni.org/a/9231-1,https://bloodhound-tracker.net/Q106178,http://www.eoas.info/biogs/P000141b,http://adb.anu.edu.au/biography/schomburgk-mor...
573,http://www.wikidata.org/entity/Q155764,Robert Brown,Brown,R.,"Brown, R.",NaN,34572082,0000 0000 8113 5572,38122,1192-1,...,1858,NaN,NaN,http://www.wikidata.org/wiki/Q155764,None,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/1192-1,https://bloodhound-tracker.net/Q155764,http://www.eoas.info/biogs/P000270b,http://adb.anu.edu.au/biography/brown-robert-1835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117560,http://www.wikidata.org/entity/Q64826644,Jim Macdougall,Macdougall,J.,"Macdougall, J.",None,NaN,NaN,NaN,NaN,...,1995,NaN,NaN,http://www.wikidata.org/wiki/Q64826644,None,None,None,None,None,http://adb.anu.edu.au/biography/macdougall-jam...
117561,http://www.wikidata.org/entity/Q79903712,Audrie Lillian McLeod,McLeod,A.L.,"McLeod, A.L.",None,NaN,NaN,NaN,NaN,...,1992,NaN,NaN,http://www.wikidata.org/wiki/Q79903712,None,None,None,None,None,http://adb.anu.edu.au/biography/mcleod-audrie-...
117562,http://www.wikidata.org/entity/Q64826630,Fred Clements,Clements,F.,"Clements, F.",None,NaN,NaN,NaN,NaN,...,1995,NaN,NaN,http://www.wikidata.org/wiki/Q64826630,None,None,None,None,None,http://adb.anu.edu.au/biography/clements-frede...
117563,http://www.wikidata.org/entity/Q64826392,Roy Deicke,Deicke,R.,"Deicke, R.",None,NaN,NaN,NaN,NaN,...,1995,NaN,NaN,http://www.wikidata.org/wiki/Q64826392,None,None,None,None,None,http://adb.anu.edu.au/biography/deicke-roy-20257


In [29]:
df.to_csv('data/wikidata_persons.csv')